<a href="https://colab.research.google.com/github/mahdi943/InceptionV3-binary/blob/master/Acc-75_class4_Training_brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io
from numpy import random

import keras
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, BatchNormalization,Activation, Flatten, GlobalAveragePooling2D, SeparableConv2D
from keras.models import Model

from keras.preprocessing import image
from keras.utils import layer_utils
from keras.layers import Dropout
from keras.regularizers import l2
from keras.regularizers import l1
np.random.seed(5)
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os
from keras.utils.np_utils import to_categorical
from PIL import Image
from keras.preprocessing.image import load_img,img_to_array,array_to_img,save_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/project/dataset/Train.zip"
!unzip "/content/drive/MyDrive/project/dataset/Test.zip"
!unzip "/content/drive/MyDrive/project/dataset/Valid.zip"

In [ ]:
y_col = ['Class0', 'Class1']

In [ ]:
from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(500, 500, 3))
base_model.layers.pop()
base_model.layers.pop()
base_model.layers.pop()
base_model.layers.pop()
for layer in base_model.layers:
  layer.trainable = False
# base_model.summary()

In [ ]:
def brain_model(base_model, img_shape = (500,500,3)):
    
    
    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=img_shape) 
    
    # Forward pass to get the output of the last pooling layer
    X = base_model(inputs)
    #X = Dropout(.5, img_shape)(X)
    X = GlobalAveragePooling2D()(X)
    X = Dropout(.25)(X)
    # Define the new binary classification head 
    X = Dense(1024, activation='relu', name='fc1')(X)
    X = Dropout(.5)(X)
    X = Dense(512, activation='relu', name='fc2')(X)
    X = Dropout(.25)(X)
    X = Dense(256, activation='relu', name='fc3')(X)
    X = Dropout(.5)(X)
    X = Dense(128, activation='relu', name='fc4')(X)
    X = Dropout(.25)(X)
    X = Dense(64, activation='relu', name='fc5')(X)
    X = Dropout(.5)(X)
        
    outputs = Dense(2,activation='softmax', name='predictions')(X)
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:

model1 = brain_model (base_model, img_shape = (500,500,3))
model2 = brain_model (base_model, img_shape = (500,500,3))
model3 = brain_model (base_model, img_shape = (500,500,3))

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset/labelTrain1.csv') 
data1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset/labelValid1.csv')
data2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset/labelTest1.csv')
data3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset/labelTestfinal.csv')


batch_size = 16

data_path = "/content/Train" 

#shuffle the data
data = data.sample(frac=1)
data.reset_index(drop = True, inplace = True)
train_aug = ImageDataGenerator(	rescale=1/255.0,	
                               rotation_range=20,	
                               zoom_range=0.05,	
                               width_shift_range=0.1,	
                               height_shift_range=0.1,	
                               shear_range=0.05,	
                               horizontal_flip=True,	
                               vertical_flip=True,	
                               fill_mode="nearest"
                               )


valid_datagen = ImageDataGenerator(rescale=1./255)
test_aug = ImageDataGenerator(rescale=1./255)

train_generator = train_aug.flow_from_dataframe(
    dataframe=data,
    directory=data_path,
    x_col='Image',
    y_col= y_col,
    class_mode="raw",
    shuffle=True,
    target_size=(500,500), 
    batch_size=batch_size)

valid_generator = test_aug.flow_from_dataframe(
    dataframe=data1,
    directory="/content/Valid",
    x_col='Image',
    y_col= y_col,
    class_mode= "raw",
    target_size=(500,500),
    batch_size=batch_size
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model1.fit(train_generator, batch_size= 16 ,epochs=10, validation_data = valid_generator, callbacks= callback)


Found 256 validated image filenames.
Found 64 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
16/16 [==============================] - 24s 1s/step - loss: 1.0310 - accuracy: 0.6172 - val_loss: 0.5766 - val_accuracy: 0.7500
Epoch 2/10
16/16 [==============================] - 17s 1s/step - loss: 0.7638 - accuracy: 0.6328 - val_loss: 0.6178 - val_accuracy: 0.7500
Epoch 3/10
16/16 [==============================] - 17s 1s/step - loss: 0.6781 - accuracy: 0.7305 - val_loss: 0.5495 - val_accuracy: 0.7500
Epoch 4/10
16/16 [==============================] - 16s 1s/step - loss: 0.6343 - accuracy: 0.6836 - val_loss: 0.5339 - val_accuracy: 0.7500
Epoch 5/10
16/16 [==============================] - 16s 1s/step - loss: 0.6245 - accuracy: 0.7148 - val_loss: 0.5623 - val_accuracy: 0.7500
Epoch 6/10
16/16 [==============================] - 16s 1s/step - loss: 0.5288 - accuracy: 0.7656 - val_loss: 0.4809 - val_accuracy: 0.9219
Epoch 7/10
16/16 [==============================] - 16s 1s/step - loss: 0.4608 - accuracy: 0.7539 - val_loss: 0.3089 - val_accuracy: 0.9219
Epoch 8/10
16/16 [==

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset-class2/labelTrain-class2.csv') 
data1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset-class2/labelValid-class2.csv')
data2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset-class2/labelTest-class2.csv')
data3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset-class2/labelTestfinal-class2.csv')


batch_size = 16

data_path = "/content/Train" 

#shuffle the data
data = data.sample(frac=1)
data.reset_index(drop = True, inplace = True)
train_aug = ImageDataGenerator(	rescale=1/255.0,	
                               rotation_range=20,	
                               zoom_range=0.05,	
                               width_shift_range=0.1,	
                               height_shift_range=0.1,	
                               shear_range=0.05,	
                               horizontal_flip=True,	
                               vertical_flip=True,	
                               fill_mode="nearest"
                               )


valid_datagen = ImageDataGenerator(rescale=1./255)
test_aug = ImageDataGenerator(rescale=1./255)

train_generator = train_aug.flow_from_dataframe(
    dataframe=data,
    directory=data_path,
    x_col='Image',
    y_col= y_col,
    class_mode="raw",
    shuffle=True,
    target_size=(500,500), 
    batch_size=batch_size)

valid_generator = test_aug.flow_from_dataframe(
    dataframe=data1,
    directory="/content/Valid",
    x_col='Image',
    y_col= y_col,
    class_mode= "raw",
    target_size=(500,500),
    batch_size=batch_size
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model2.fit(train_generator, batch_size= 16 ,epochs=10, validation_data = valid_generator, callbacks= callback)


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset-class3/labelTrain-class3.csv') 
data1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset-class3/labelValid-class3.csv')
data2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset-class3/labelTest-class3.csv')
data3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS515 - project/dataset/dataset-class3/labelTestfinal-class3.csv')


batch_size = 16

data_path = "/content/Train" 

#shuffle the data
data = data.sample(frac=1)
data.reset_index(drop = True, inplace = True)
train_aug = ImageDataGenerator(	rescale=1/255.0,	
                               rotation_range=20,	
                               zoom_range=0.05,	
                               width_shift_range=0.1,	
                               height_shift_range=0.1,	
                               shear_range=0.05,	
                               horizontal_flip=True,	
                               vertical_flip=True,	
                               fill_mode="nearest"
                               )


valid_datagen = ImageDataGenerator(rescale=1./255)
test_aug = ImageDataGenerator(rescale=1./255)

train_generator = train_aug.flow_from_dataframe(
    dataframe=data,
    directory=data_path,
    x_col='Image',
    y_col= y_col,
    class_mode="raw",
    shuffle=True,
    target_size=(500,500), 
    batch_size=batch_size)

valid_generator = test_aug.flow_from_dataframe(
    dataframe=data1,
    directory="/content/Valid",
    x_col='Image',
    y_col= y_col,
    class_mode= "raw",
    target_size=(500,500),
    batch_size=batch_size
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model3.fit(train_generator, batch_size= 16 ,epochs=20, validation_data = valid_generator, callbacks= callback)


In [ ]:
if model1.predict == 1:
  # do somthing
  x = 1
elif model2.predict == 1:
  # do somthing
  x = 2
elif model3.predict == 1:
  # do somthing
  x = 3
else:
  # do somthing
  x = 4



In [ ]:
test_generator = test_aug.flow_from_dataframe(
    dataframe=data2,
    directory="/content/Test",
    x_col='Image',
    y_col= y_col,
    shuffle=True,
    class_mode= "raw",
    target_size=(500,500),
    batch_size=batch_size
)

Found 80 validated image filenames.


In [ ]:
y_predict = model1.predict(test_generator)
print(type(y_predict))


<class 'numpy.ndarray'>


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

y_predict = model1.predict(test_generator)

y_predict = y_predict.argmax(-1)
accuracy_score(data3['label'],y_predict)
    


0.6625

In [ ]:
# plt.plot(history.history['loss'])
# plt.title('model loss')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.show()

In [ ]:
# plt.plot(history.history['accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.show()